In [42]:
import pandas as pd
import numpy as np
import twint
import json
import openai
from textblob import TextBlob
import nest_asyncio
nest_asyncio.apply()

default_keywords=["trump", "impeachment", "bachelor", "covid"]
elonmusk = ['dogecoin', 'POTUS', 'cleantechnica', 'TheOnion', 'TheBabylonBee', 'karpathy', 'Astro_Soichi', 'PopMech', 'PyTorch', 'Nigel_Lockyer', 'jagarikin', 'AstroVicGlover', 'Grimezsz', 'TashaARK']
satyanadella = ['Herbert_Diess', 'ChrstnKlein', 'vasujakkal', 'amandaksilver', 'nicoledezen', 'KingJames', 'drhew', 'panos_panay', 'youngdchris', 'MicrosoftWomen']
stephencurry = ['BryceCash6', 'Patty_Mills', 'RealDealBeal23', 'gusjohnson', 'arneduncan', 'PatrikFrisk', 'iamcarljones', 'zlurie', 'TSM', 'QCook323']
kamalaharris = ['LaCasaBlanca', 'SecondGentleman', 'FLOTUS', 'POTUS', 'WhiteHouse', 'SenatorHick', 'RepBowman', 'RepRitchie', 'AlexPadilla4CA', 'AstroAnnimal', 'CASOSvote', 'CAPAction', 'RobBontaCA']
openai.api_key = "sk-8xLMvhl5qyjjmajkSRQeEuggYQTdspJViTCInd9Y"

def get_tweets(keyword, num_posts, likes, retweets):
    c = twint.Config()
    c.Limit = num_posts
    if(keyword.strip()):
        c.Search = keyword
        c.Language = "en"
        c.Pandas = True
        c.Store_object = False
        c.Hide_output = True
        c.User_full = True
        c.Min_likes = likes
        c.Min_retweets = retweets
        twint.run.Search(c)
        tweets = twint.storage.panda.Tweets_df
        return tweets
    else:
        individual_tweets = []
        for keyword in default_keywords:
            c.Search = keyword
            c.Language = "en"
            c.Pandas = True
            c.Store_object = False
            c.Hide_output = True
            c.User_full = True
            c.Min_likes = likes
            c.Min_retweets = retweets
            twint.run.Search(c)
            tweets = twint.storage.panda.Tweets_df
            individual_tweets.append(tweets)
        all_tweets = pd.concat(individual_tweets)
        return all_tweets
    
def get_controversial(keyword, num_posts, likes, retweets):
    textblob = get_tweets(keyword, num_posts, likes, retweets)
    textblob['tweet'] = textblob['tweet'].astype(str)
    polarity = lambda x: TextBlob(x).sentiment.polarity
    subjectivity = lambda x: TextBlob(x).sentiment.subjectivity
    textblob['polarity'] = textblob['tweet'].apply(polarity)
    textblob['subjectivity'] = textblob['tweet'].apply(subjectivity)

    controversial_tweets = textblob[textblob['polarity'] > 0.5]
    return controversial_tweets

def get_explained(keyword, num_posts, likes, retweets):
    tweets = get_tweets(keyword, num_posts, likes, retweets)
    is_explanation = []
    for i in range(0, len(tweets)):
        tweettxt = tweets['tweet'][i]
        if 'explain' in tweettxt:
            is_explanation.append(True)
        else: 
            is_explanation.append(False)
    tweets['is_explanation'] = is_explanation
    explained_tweets = tweets[tweets['is_explanation'] == True]
    return explained_tweets

def get_verified(keyword, num_posts, likes, retweets):
    c = twint.Config()
    c.Limit = num_posts
    if (keyword.strip()):
        c.Search = keyword
        c.Language = "en"
        c.Verified = True
        c.Pandas = True
        c.Store_object = False
        c.Hide_output = True
        c.User_full = True
        c.Min_likes = likes
        c.Min_retweets = retweets
        twint.run.Search(c)
        verified_tweets = twint.storage.panda.Tweets_df
        return verified_tweets
    else:
        individual_tweets = []
        for keyword in default_keywords:
            c.Search = keyword
            c.Language = "en"
            c.Pandas = True
            c.Store_object = False
            c.Hide_output = True
            c.User_full = True
            c.Verified = True
            c.Min_likes = likes
            c.Min_retweets = retweets
            twint.run.Search(c)
            tweets = twint.storage.panda.Tweets_df
            individual_tweets.append(tweets)
        all_tweets = pd.concat(individual_tweets)
        return all_tweets 

def get_viral(keyword, num_posts):
    c = twint.Config()
    c.Limit = num_posts
    c.Language = "en"
    c.Min_likes = 50000
    c.Min_retweets = 1000
    c.Min_replies = 1000
    if(keyword.strip()):
        c.Search = keyword
        c.Pandas = True
        c.Store_object = False
        c.Hide_output = True
        c.User_full = True
        twint.run.Search(c)
        viral_tweets = twint.storage.panda.Tweets_df
        return viral_tweets
    else:
        individual_tweets = []
        for keyword in default_keywords:
            c.Search = keyword
            c.Pandas = True
            c.Store_object = False
            c.Hide_output = True
            c.User_full = True
            twint.run.Search(c)
            tweets = twint.storage.panda.Tweets_df
            individual_tweets.append(tweets)
        all_tweets = pd.concat(individual_tweets)
        return all_tweets 

def get_user_tweets(username, keyword, num_posts):
    c = twint.Config()
    c.Pandas = True
    c.Search = keyword
    c.Store_object = False
    c.Hide_output = True
    c.Limit = num_posts
    c.Username = username
    twint.run.Search(c)
    tweets = twint.storage.panda.Tweets_df
    return tweets
    
    
def get_influencer_feed(username, keyword, num_posts):
    following = []
    if username == 'elonmusk':
        following = elonmusk
    elif username == 'satyanadella':
        following = satyanadella
    elif username == 'StephenCurry30':
        following = stephencurry
    elif username == 'VP':
        following = kamalaharris

    individual_tweets = []
    for user in following:
        tweets = get_user_tweets(user, keyword, num_posts)
        individual_tweets.append(tweets)
    all_tweets = pd.concat(individual_tweets)
    return all_tweets

def get_openai_positive(keyword, num_posts, likes, retweets):
    tweets = get_tweets(keyword, num_posts, likes, retweets)
    is_positive = []
    for i in range(0, len(tweets)):
        tweettxt = tweets['tweet'][i]
        response = openai.Completion.create(
          engine="davinci",
          prompt=f"Social media post: \"{tweettxt}\"\nSentiment (positive, neutral, negative):",
          temperature=0,
          max_tokens=1,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        sentiment = response["choices"][0]["text"]
        if ('positive' in sentiment.lower()):
            is_positive.append(True)
        else:
            is_positive.append(False)
    tweets['is_positive'] = is_positive
    positive_tweets = tweets[tweets['is_positive'] == True]
    return positive_tweets

def get_openai_negative(keyword, num_posts, likes, retweets):
    tweets = get_tweets(keyword, num_posts, likes, retweets)
    is_negative = []
    for i in range(0, len(tweets)):
        tweettxt = tweets['tweet'][i]
        response = openai.Completion.create(
          engine="davinci",
          prompt=f"Social media post: \"{tweettxt}\"\nSentiment (positive, neutral, negative):",
          temperature=0,
          max_tokens=1,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        sentiment = response["choices"][0]["text"]
        if ('negative' in sentiment.lower()):
            is_negative.append(True)
        else:
            is_negative.append(False)
    tweets['is_negative'] = is_negative
    negative_tweets = tweets[tweets['is_negative'] == True]
    return negative_tweets

def get_openai_neutral(keyword, num_posts, likes, retweets):
    tweets = get_tweets(keyword, num_posts, likes, retweets)
    is_neutral = []
    for i in range(0, len(tweets)):
        tweettxt = tweets['tweet'][i]
        response = openai.Completion.create(
          engine="davinci",
          prompt=f"Social media post: \"{tweettxt}\"\nSentiment (positive, neutral, negative):",
          temperature=0,
          max_tokens=1,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        sentiment = response["choices"][0]["text"]
        if ('neutral' in sentiment.lower()):
            is_neutral.append(True)
        else:
            is_neutral.append(False)
    tweets['is_neutral'] = is_neutral
    neutral_tweets = tweets[tweets['is_neutral'] == True]
    return neutral_tweets

def get_tweets_using_algorithm(algorithm, keyword, likes=100, retweets=50):
    num_posts = 100
    if('controversial' in algorithm):
        tweets = get_controversial(keyword, num_posts, likes, retweets)
    elif('explained' in algorithm):
        tweets = get_explained(keyword, 1000, 0, 0)
    elif('influencer' in algorithm):
        _, username = algorithm.split("-", 1)
        tweets = get_influencer_feed(username, keyword, num_posts)
    elif('viral' in algorithm):
        tweets = get_viral(keyword, 100)
    elif('verified' in algorithm):
        tweets = get_verified(keyword, 100, likes, retweets)
    elif('positive' in algorithm):
        tweets = get_openai_positive(keyword, 100, likes, retweets)
    elif('negative' in algorithm):
        tweets = get_openai_negative(keyword, 100, likes, retweets)
    elif('neutral' in algorithm):
        tweets = get_openai_neutral(keyword, 100, likes, retweets)
    else:
        tweets = pd.DataFrame()
    filtered = tweets[['conversation_id', 'created_at', 'tweet', 'user_id', 'username', 'name', 'nlikes', 'nreplies', 'nretweets']]
    filtered.reset_index(drop=True, inplace=True)
    result = filtered.to_json(orient="index")
    parsed = list(json.loads(result).values())
    json_tweets = json.dumps(parsed, indent=4)
    return json_tweets

json_tweets = get_tweets_using_algorithm('neutral', 'bachelor')
print(json_tweets)

[]


In [ ]:
influencer_tweets = get_tweets_using_algorithm('influencer-elonmusk', '')
controversial_tweets = get_tweets_using_algorithm('controversial', 'trump')
explained_tweets = get_tweets_using_algorithm('explained', 'bachelor')
verified_tweets = get_tweets_using_algorithm('verified', 'impeach')
viral_tweets = get_tweets_using_algorithm('viral', 'impeach')
positive_tweets = get_tweets_using_algorithm('positive', '')
negative_tweets = get_tweets_using_algorithm('negative', '')
neutral_tweets = get_tweets_using_algorithm('neutral', '')

#explained no keyword, positive/negative/neutral no keyword